In [1]:
import re
import numpy
import pandas

In [2]:
with open('./metabolic_network_v2.txt', 'r') as infile:
    data = pandas.read_csv(infile, delimiter = '\t', header = None)
data

,0,1,2,3
0,spontaneous,LACTOSE-MUTAROTATION,alpha-lactose,beta-lactose
1,spontaneous,GALACTOSE-MUTAROTATION,alpha-GALACTOSE,beta-GALACTOSE
2,spontaneous,GLUCOSE-MUTAROTATION,alpha-glucose,beta-glucose
3,lacY,TRANS-RXN-24,"PER-PROTON, PER-alpha-lactose","PROTON, alpha-lactose"
4,lacY,TRANS-RXN-24-beta,"PER-PROTON, PER-beta-lactose","PROTON, beta-lactose"
5,lacY,TRANS-RXN-94,"PER-PROTON, PER-MELIBIOSE","PROTON, MELIBIOSE"
6,lacY,RXN0-7215,"PER-PROTON, PER-CPD-3561","PROTON, CPD-3561"
7,lacY,RXN0-7217,"PER-PROTON, PER-CPD-3785","PROTON, CPD-3785"
8,lacY,RXN-17755,"PER-PROTON, PER-CPD-3801","PROTON, CPD-3801"
9,lacZ,BETAGALACTOSID-RXN,"beta-lactose, WATER","beta-GALACTOSE, beta-glucose"


In [3]:
for rxn in data.values:
    if 'CPLX' in rxn[0]:
        enzyme = 'cplx(name = \'{:s}\')'.format(rxn[0].replace('-', '_'))
    else:
        enzyme = 'prot(name = \'{:s}\')'.format(rxn[0].replace('-', '_'))
        
    name = rxn[1].replace('-', '_')
    substrates = rxn[2].replace('-', '_').split(', ')
    products = rxn[3].replace('-', '_').split(', ')

    LHS = []
    RHS = []
    
    for subs in substrates:
        if subs[0].isdigit():
            subs = '_' + subs
        
        if 'PER' in subs:
            LHS.append('met(name = \'{:s}\', loc = \'per\')'.format(subs.replace('PER_', '')))
        else:
            LHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(subs))

    for prod in products:
        if prod[0].isdigit():
            prod = '_' + prod
        
        if 'PER' in prod:
            RHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(prod))
        else:
            RHS.append('met(name = \'{:s}\', loc = \'cyt\')'.format(prod))
            
    LHS = ' + '.join(LHS)
    RHS = ' + '.join(RHS)
        
    if rxn[0] == 'spontaneous':
        Rule = 'Rule(\'{:s}\', {:s} |\n'\
               '    {:s}, \n' \
               '    Parameter(\'fwd_{:s}\', 1), \n' \
               '    Parameter(\'rvs_{:s}\', 1))' \
               .format(name, LHS, RHS, name, name)
    
    else: # need an enzyme
        Rule = 'Rule(\'{:s}\', {:s} + {:s} | \n'\
               '    {:s} + {:s}, \n' \
               '    Parameter(\'fwd_{:s}\', 1), \n' \
               '    Parameter(\'rvs_{:s}\', 0))' \
               .format(name, enzyme, LHS, enzyme, RHS, name, name)
        
    print(Rule)
    print()

Rule('LACTOSE_MUTAROTATION', met(name = 'alpha_lactose', loc = 'cyt') |
    met(name = 'beta_lactose', loc = 'cyt'), 
    Parameter('fwd_LACTOSE_MUTAROTATION', 1), 
    Parameter('rvs_LACTOSE_MUTAROTATION', 1))

Rule('GALACTOSE_MUTAROTATION', met(name = 'alpha_GALACTOSE', loc = 'cyt') |
    met(name = 'beta_GALACTOSE', loc = 'cyt'), 
    Parameter('fwd_GALACTOSE_MUTAROTATION', 1), 
    Parameter('rvs_GALACTOSE_MUTAROTATION', 1))

Rule('GLUCOSE_MUTAROTATION', met(name = 'alpha_glucose', loc = 'cyt') |
    met(name = 'beta_glucose', loc = 'cyt'), 
    Parameter('fwd_GLUCOSE_MUTAROTATION', 1), 
    Parameter('rvs_GLUCOSE_MUTAROTATION', 1))

Rule('TRANS_RXN_24', prot(name = 'lacY') + met(name = 'PROTON', loc = 'per') + met(name = 'alpha_lactose', loc = 'per') | 
    prot(name = 'lacY') + met(name = 'PROTON', loc = 'cyt') + met(name = 'alpha_lactose', loc = 'cyt'), 
    Parameter('fwd_TRANS_RXN_24', 1), 
    Parameter('rvs_TRANS_RXN_24', 0))

Rule('TRANS_RXN_24_beta', prot(name = 'lacY') + 